In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np

data_path = '/content/drive/MyDrive/DACON 경진대회/제주 특산물가격 예측/data/'

df = pd.read_csv(f"{data_path}submission_autogluon_ver2.csv")
train_df = pd.read_csv(f"{data_path}train.csv")

In [16]:
train_df.rename(columns={'supply(kg)' : 'supply', 'price(원/kg)' : 'price'}, inplace=True)
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
min_df = train_df[train_df['price'] != 0].groupby(['item', 'corporation', 'location', 'month'])['price'].min().reset_index()
min_df = min_df[min_df['month'] == 3]
min_df.drop(columns=['month'], inplace=True)
min_df = min_df.reset_index(drop=True)
min_df

,item,corporation,location,price
0,BC,A,J,1796.0
1,BC,A,S,1597.0
2,BC,B,J,1120.0
3,BC,B,S,2000.0
4,BC,C,J,205.0
5,BC,C,S,2250.0
6,BC,D,J,1399.0
7,BC,E,J,1499.0
8,BC,E,S,922.0
9,CB,A,J,206.0


In [17]:
df['item'] = df.ID.str[0:2]
df['corporation'] = df.ID.str[3:4]
df['location'] = df.ID.str[5:6]
df

,ID,answer,item,corporation,location
0,TG_A_J_20230304,3242.523713,TG,A,J
1,TG_A_J_20230305,389.450981,TG,A,J
2,TG_A_J_20230306,3255.184581,TG,A,J
3,TG_A_J_20230307,3395.988407,TG,A,J
4,TG_A_J_20230308,3314.301545,TG,A,J
...,...,...,...,...,...
1087,RD_F_J_20230327,513.469446,RD,F,J
1088,RD_F_J_20230328,515.985473,RD,F,J
1089,RD_F_J_20230329,516.326125,RD,F,J
1090,RD_F_J_20230330,509.636177,RD,F,J


In [18]:
df = df.merge(min_df, on=['item', 'corporation', 'location'], how='left')
df

,ID,answer,item,corporation,location,price
0,TG_A_J_20230304,3242.523713,TG,A,J,854.0
1,TG_A_J_20230305,389.450981,TG,A,J,854.0
2,TG_A_J_20230306,3255.184581,TG,A,J,854.0
3,TG_A_J_20230307,3395.988407,TG,A,J,854.0
4,TG_A_J_20230308,3314.301545,TG,A,J,854.0
...,...,...,...,...,...,...
1087,RD_F_J_20230327,513.469446,RD,F,J,244.0
1088,RD_F_J_20230328,515.985473,RD,F,J,244.0
1089,RD_F_J_20230329,516.326125,RD,F,J,244.0
1090,RD_F_J_20230330,509.636177,RD,F,J,244.0


In [19]:
df.loc[ df['answer'] < df['price'], 'answer'] = 0.0
df

,ID,answer,item,corporation,location,price
0,TG_A_J_20230304,3242.523713,TG,A,J,854.0
1,TG_A_J_20230305,0.000000,TG,A,J,854.0
2,TG_A_J_20230306,3255.184581,TG,A,J,854.0
3,TG_A_J_20230307,3395.988407,TG,A,J,854.0
4,TG_A_J_20230308,3314.301545,TG,A,J,854.0
...,...,...,...,...,...,...
1087,RD_F_J_20230327,513.469446,RD,F,J,244.0
1088,RD_F_J_20230328,515.985473,RD,F,J,244.0
1089,RD_F_J_20230329,516.326125,RD,F,J,244.0
1090,RD_F_J_20230330,509.636177,RD,F,J,244.0


In [20]:
df.drop(columns=['item', 'corporation', 'location', 'price'], inplace=True)
df

,ID,answer
0,TG_A_J_20230304,3242.523713
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3255.184581
3,TG_A_J_20230307,3395.988407
4,TG_A_J_20230308,3314.301545
...,...,...
1087,RD_F_J_20230327,513.469446
1088,RD_F_J_20230328,515.985473
1089,RD_F_J_20230329,516.326125
1090,RD_F_J_20230330,509.636177


In [21]:
df.to_csv(f"{data_path}test_autogluon_ver2_.csv", index=False)